In [133]:
import numpy as np
from qiskit import QuantumCircuit, ClassicalRegister, QuantumRegister
from qiskit import BasicAer, execute

## Circuit Creation

### Preperation
Prepares a Bell-state by performing ($H\otimes I$) on the first qubit and entangling it with the second qubit.

In [134]:
encodeQ1 = QuantumRegister(1, 'encodeQ1')
encodeQ2 = QuantumRegister(1, 'encodeQ2')

preperationCircuit = QuantumCircuit(encodeQ1, encodeQ2)
preperationCircuit.h(encodeQ1)
preperationCircuit.cx(encodeQ1, encodeQ2)
preperationCircuit

### Encode
Encodes the message into the first Qubit. This is done via a controlled $X$ (NOT) Gate and $Z$ (Phase) Gate based on the message. This rotation changes the Bell-state such that when the qubit is sent, it can be recombined with the other encoding qubit via a controlled $X$ Gate to retrieve the message. 

In [135]:
encodedMessage = ClassicalRegister(2, 'em')
messageQubits = QuantumRegister(2, 'cm')

encodeCircuit = QuantumCircuit(encodedMessage, messageQubits, encodeQ1)

# Get sent classical message
encodeCircuit.measure(messageQubits, encodedMessage)

# Encoding
encodeCircuit.cx(messageQubits[1], encodeQ1)
encodeCircuit.cz(messageQubits[0], encodeQ1)
encodeCircuit

### Transfer
This swap operation represents a transfer of quantum information from one information system to another.

In [136]:
swapQubit = QuantumRegister(1, 'swap')

transferCircuit = QuantumCircuit(encodeQ1, swapQubit)
transferCircuit.swap(encodeQ1, swapQubit)
transferCircuit

### Decode
Decode the message using the prepared qubit and the newly arrived qubit.

In [137]:
decodeCircuit = QuantumCircuit(swapQubit, encodeQ2)
decodeCircuit.cx(swapQubit, encodeQ2)
decodeCircuit.h(swapQubit)

In [138]:
receivingRegisters = ClassicalRegister(2, 'recreg')

readCircuit = QuantumCircuit(swapQubit, encodeQ2, receivingRegisters)
readCircuit.measure(swapQubit[0], receivingRegisters[0])
readCircuit.measure(encodeQ2[0], receivingRegisters[1])
readCircuit

## Visualization

### Preperation Analysis

In [139]:
preperationCircuit.draw()

In [140]:
encodeCircuit.draw()

In [141]:
transferCircuit.draw()

In [142]:
decodeCircuit.draw()

In [143]:
readCircuit.draw()

In [149]:
allCircuits = (preperationCircuit + encodeCircuit + transferCircuit + decodeCircuit + readCircuit)
allCircuits.count_ops()

{'measure': 4, 'h': 2, 'cx': 3, 'cz': 1, 'swap': 1}

In [153]:
(encodeCircuit + (decodeCircuit + readCircuit)).num_tensor_factors()

2

In [145]:
backend_sim = BasicAer.get_backend('qasm_simulator')
result = execute(allCircuits, backend_sim).result()
print(result)
counts = result.get_counts(allCircuits)
print(counts)

Result(backend_name='qasm_simulator', backend_version='2.0.0', header=Obj(backend_name='qasm_simulator'), job_id='54eceac8-2fc6-45cc-8371-42bf8d626ddf', qobj_id='8f48d5ba-0c22-40b5-8bbf-2dd8dd6d0382', results=[ExperimentResult(data=ExperimentResultData(counts=Obj(0x0=1024)), header=Obj(clbit_labels=[['em', 0], ['em', 1], ['recreg', 0], ['recreg', 1]], compiled_circuit_qasm='OPENQASM 2.0;\ninclude "qelib1.inc";\nqreg encodeQ1[1];\nqreg encodeQ2[1];\nqreg cm[2];\nqreg swap[1];\ncreg em[2];\ncreg recreg[2];\nmeasure cm[1] -> em[1];\nmeasure cm[0] -> em[0];\nu2(0,pi) encodeQ1[0];\ncx encodeQ1[0],encodeQ2[0];\ncx cm[1],encodeQ1[0];\nu2(0,pi) encodeQ1[0];\ncx cm[0],encodeQ1[0];\nu2(0,pi) encodeQ1[0];\ncx encodeQ1[0],swap[0];\ncx swap[0],encodeQ1[0];\ncx encodeQ1[0],swap[0];\ncx swap[0],encodeQ2[0];\nu2(0,pi) swap[0];\nmeasure swap[0] -> recreg[0];\nmeasure encodeQ2[0] -> recreg[1];\n', creg_sizes=[['em', 2], ['recreg', 2]], memory_slots=4, n_qubits=5, name='circuit97', qreg_sizes=[['encodeQ1